In [ ]:
import pandas as pd
import os

dfHotels = pd.DataFrame()

for filename in os.listdir('fragments'):
    if filename.endswith('.csv') and 'without' in filename:
        file = 'fragments/' + filename
        dfHotels = dfHotels.append(pd.read_csv(file, sep=',', encoding='utf-8'), sort=False)
        continue

        
dfHotels = dfHotels.drop_duplicates(subset=['uri'], keep='first')
dfHotels = dfHotels.drop_duplicates(subset=['source_reduced', 'Hotel/name'], keep='first')
display(dfHotels)

In [ ]:
header = []
for name in dfHotels.columns:
    name = name.replace('Hotel/', 'hotel_').replace('PostalAddress/', 'postaladdress_').replace('Place/', 'place_')
    header.append(name)

dfHotels.columns = header

del dfHotels['<http://purl.org/dc/terms/title>']
del dfHotels['<http://schema.org/AggregateRating/bestRating>']
del dfHotels['<http://schema.org/AggregateRating/ratingValue>']
del dfHotels['<http://schema.org/hotel_address>']
del dfHotels['<http://schema.org/hotel_aggregateRating>']
#del dfHotels['<http://schema.org/hotel_image>']
del dfHotels['<http://schema.org/hotel_name>']
del dfHotels['<http://schema.org/hotel_priceRange>']
del dfHotels['<http://schema.org/postaladdress_addressCountry>']
del dfHotels['<http://schema.org/postaladdress_addressLocality>']
del dfHotels['<http://schema.org/postaladdress_streetAddress>']
del dfHotels['<http://schema.org/postaladdress_postalCode>']
del dfHotels['<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>']
del dfHotels['<http://www.w3.org/1999/xhtml/microdata#item>']
#del dfHotels['<http://schema.org/hotel_sameAs>']
del dfHotels['<http://schema.org/hotel_url>']


In [ ]:
dfHotelsNew = pd.DataFrame()
replace_names = ['www.hotels.com', 'au.hotels.com', 'uk.hotels.com', 'in.hotels.com', 'ie.hotels.com',
                 'nz.hotels.com', 'sg.hotels.com', 'ca.hotels.com', 'za.hotels.com', 'ph.hotels.com']
for index, row in dfHotels.iterrows():
    if row['source_reduced'] in replace_names:
        row['source_reduced'] = 'hotels.com'
    
    dfHotelsNew = dfHotelsNew.append(row)
    
dfHotels = dfHotelsNew

In [ ]:
dfHotels = dfHotels.drop_duplicates(subset=['source_reduced', 'hotel_name'], keep='first')
len(dfHotels)

In [ ]:
# Reduce Hotels
dfHotels = dfHotels[~dfHotels['hotel_name'].str.lower().str.contains('hilton')]
dfHotels = dfHotels[dfHotels['hotel_name'].str.lower().str.contains('paris')]
len(dfHotels)

In [ ]:
import csv

listOthers = []

def splitDataSet(name):
    dfHotelsCoins = pd.DataFrame()
    counter = 0
    dfHotelsSubset = dfHotels[dfHotels['source_reduced'] == name]
    display(name)
    for index, row in dfHotelsSubset.iterrows():
        row['uri'] = name + '_' + str(counter)
        counter = counter + 1
        dfHotelsCoins = dfHotelsCoins.append(row)
    filename = 'subsets/hotels_' + name + '.csv'
    dfHotelsCoins.to_csv(filename, sep=',', encoding='utf-8', index=False, quotechar='"', quoting=csv.QUOTE_ALL)

from nltk.metrics import jaccard_distance
from nltk import wordpunct_tokenize

def checkExectMatch(name):
    dfHotelsSubset = dfHotels[dfHotels['source_reduced'] == name]
    dfHotelsOther = dfHotels[dfHotels['source_reduced'] != name]
    
    size = len(dfHotelsSubset)
    
    if size > 10:
        splitDataSet(name)
        display(name)
        subset = dfHotelsOther[dfHotelsOther['hotel_name'].isin(dfHotelsSubset['hotel_name'].values)]['source_reduced'].unique()
        display(subset)
        display(size)
    else:
        listOthers.append(name)

sites = ['revngo.com', 'www.nighttours.com', 'www.touristlink.com', 'www.ihg.com']
#checkExectMatch('london.carpediem.cd')
[checkExectMatch(name) for name in sites]
#checkExectMatch('www.hotels.com')

In [ ]:
#Write Others to file
dfHotelsCoins = pd.DataFrame()
dfHotelsSubset = dfHotels[dfHotels['source_reduced'].isin(listOthers)]
counter = 0

for index, row in dfHotelsSubset.iterrows():
    row['uri'] = 'other_' + str(counter)
    counter = counter + 1
    dfHotelsCoins = dfHotelsCoins.append(row)
filename = 'subsets/hotels_other.csv'
dfHotelsCoins.to_csv(filename, sep=',', encoding='utf-8', index=False, quotechar='"', quoting=csv.QUOTE_ALL)
    
